**Given**: Two positive integers k (k≤7k≤7) and N (N≤2kN≤2k). In this problem, we begin with Tom, who in the 0th generation has genotype Aa Bb. Tom has two children in the 1st generation, each of whom has two children, and so on. Each organism always mates with an organism having genotype Aa Bb.

**Return**: The probability that at least N Aa Bb organisms will belong to the k-th generation of Tom's family tree (don't count the Aa Bb mates at each level). Assume that Mendel's second law holds for the factors.

Sample Dataset
2 1

Sample Output
0.684

**Solution**

Punnett square for mating organisms with AaBb genotype:

AaBb x AaBb = (AB | Ab | aB | ab) x  (AB | Ab | aB | ab) = 

AABB + AABb + AaBB + AaBb +

AABb + AAbb + AaBb + Aabb +

AaBB + AaBb + aaBB + aaBb +

AaBb + Aabb + aaBb + aabb =

1x AABB +

2x AABb +

1x AAbb +

2x AaBB +

4x AaBb +

2x Aabb +

1x aaBb +

2x aaBb +

1x aabb


Using Mendel's Second Law, alleles for different factors are inherited with no dependence on each other.

Aa x Aa = (A | a) x (A | a) = AA + Aa + aA + aa = 1x AA + 2x Aa + 1x aa

Bb x Bb = (B | b) x (B | b) = BB + Bb + bB + bb = 1x BB + 2x Bb + 1x bb

Multiplying these 2 results, (1x AA + 2x Aa + 1x aa) and (1x BB + 2x Bb + 1x bb), will yield the same result as above ro AaBb x AaBb.


**Hint**

Prob('AaBb' in F1) = 4/16 (see above) = 1/4 = Prob('Aa' in F1) * Prob('Bb' in F1)

In [116]:
from tools import punnett_square
from copy import deepcopy
import math

In [101]:
def mating(genotype1, genotype2):
    """
    Args:
        + genotype1,2 : two tuples of dictionaries {str: float}, resulting from Punnett square calculation. 
                        The number of elements in every tuple is equal to the number of factors in genotype (2 for AaBb).
    
    Returns:
        one tuple of dictionaries {str: float}, 1 dictionary per factor
    """
    #print("mating", genotype1, "with", genotype2)
    offspring_genotype = tuple()
    # Multiplying independently parts of genotype dictionary, factor by factor
    n_factors = len(genotype1)
    for i in range(n_factors):
        d = multiply_one_factor_dict(genotype1[i], genotype2[i])
        offspring_genotype += (d,)
    return offspring_genotype

In [102]:
def multiply_one_factor_dict(punnett1, punnett2):
    """
    Args:
        + punnett1,2: {str: float}
                        Key:   2-allele combinaton
                        Value: probability of observing this combination
    Returns:
        + {str: float} the same format that input dictionaries
    """
    product = dict()
    # Keys of each punnett dictionary could take values 'AA', 'Aa', 'aa'.
    for key1 in punnett1: 
        for key2 in punnett2:
            coeff = punnett1[key1] * punnett2[key2]
            p_temp = punnett_square(key1, key2)
            for d_key in p_temp:
                if d_key not in product:
                    product[d_key] = coeff * p_temp[d_key]
                else:
                    product[d_key] += coeff * p_temp[d_key]
    return product

In [108]:
def make_family_tree(k):
    """
    Mating process for `k` generations.
    """
    AaBb_genotype = ({'Aa': 1}, {'Bb': 1})
    parent = deepcopy(AaBb_genotype)
    n_children = 2

    family_tree = {0: [parent]}
    for generation_index in range(1,k+1):
        new_generation = list()
        #print("\n\ngeneration #", generation_index)
        for now_adult in family_tree[generation_index - 1]:
            for i in range(n_children):
                child = mating(genotype0, AaBb_genotype)
                new_generation.append(child)
        #print("new generation:", new_generation)
        family_tree[generation_index] = new_generation
    return family_tree

In [182]:
tree = make_family_tree(4)
tree[2]

[({'AA': 0.25, 'Aa': 0.5, 'aa': 0.25}, {'BB': 0.25, 'Bb': 0.5, 'bb': 0.25}),
 ({'AA': 0.25, 'Aa': 0.5, 'aa': 0.25}, {'BB': 0.25, 'Bb': 0.5, 'bb': 0.25}),
 ({'AA': 0.25, 'Aa': 0.5, 'aa': 0.25}, {'BB': 0.25, 'Bb': 0.5, 'bb': 0.25}),
 ({'AA': 0.25, 'Aa': 0.5, 'aa': 0.25}, {'BB': 0.25, 'Bb': 0.5, 'bb': 0.25})]

+ Probability that AT LEAST N orgamisms in the generation  are **AaBb** = 1 - P(less than N organisms are **AaBb**) = 1 - P(0 x **AaBb**) - P(1 x **AaBb**) - ... - P(N-1 x **AaBb**)


+ P(Exactly N organisms are **AaBb**) = P(Exactly N are **Aa**) **x** P(Exactly N are **Bb**)

In [175]:
def P_at_least_N_organisms_have_given_genotype(N, generation, genotype):
    p_less_than_N = 0
    for i in range(N):
        p_i = P_exactly_N_organisms_have_given_genotype(i, generation, genotype)
        print("exactly", i, ":", p_i)
        p_less_than_N += p_i
    return 1.0 - p_less_than_N

        
def P_exactly_N_organisms_have_given_genotype(N, generation, genotype):
    final_proba = 1.0
    m = len(generation)
    child_punnett = generation[0]
    
    # Probability of a genotype to be equal to the target genotype
    # For this, ALL of its factors are equal to corresponding factors of the target genotype
    yes_proba = 1.0
    for factor_index in range(len(genotype)):
        factor = genotype[factor_index]
        yes_proba *= child_punnett[factor_index][factor]
    # Probability of a genotype to be not equal to the target genotype
    no_proba = 1.0 - yes_proba
       
    # YES for the first `N`
    for i in range(N):
        final_proba *= yes_proba
    # NO for the next `m-N`
    for j in range(N, m):
        final_proba *= no_proba
    
    final_proba *= nCr(m, N)
    return final_proba

def nCr(n, r):
    return int(math.factorial(n) / (math.factorial(r)*math.factorial(n-r)))

In [179]:
def solve(k, N):
    tree = make_family_tree(k)
    last_generation = tree[k]
    print(len(last_generation), "organisms in the last generation")
    return P_at_least_N_organisms_have_given_genotype(N, last_generation, ('Aa','Bb'))

In [180]:
# Test
solve(k=3, N=2)

8 organisms in the last generation
exactly 0 : 0.1001129150390625
exactly 1 : 0.2669677734375


0.6329193115234375

In [181]:
# Submission
solve(k=7, N=35)

128 organisms in the last generation
exactly 0 : 1.0182202130902541e-16
exactly 1 : 4.344406242518418e-15
exactly 2 : 9.195659879997318e-14
exactly 3 : 1.2873923831996245e-12
exactly 4 : 1.3410337324996089e-11
exactly 5 : 1.1085878855330101e-10
exactly 6 : 7.575350551142237e-10
exactly 7 : 4.400917939235013e-09
exactly 8 : 2.2187961276976525e-08
exactly 9 : 9.86131612310068e-08
exactly 10 : 3.9116553954966027e-07
exactly 11 : 1.3987131414199972e-06
exactly 12 : 4.54581770961499e-06
exactly 13 : 1.3520893700393306e-05
exactly 14 : 3.7021494655838815e-05
exactly 15 : 9.378778646145832e-05
exactly 16 : 0.00022079208062801648
exactly 17 : 0.0004848767260850557
exactly 18 : 0.0009966910480637258
exactly 19 : 0.0019234388646843829
exactly 20 : 0.003494247270843296
exactly 21 : 0.005990138178588506
exactly 22 : 0.009711284622863183
exactly 23 : 0.014918785072804315
exactly 24 : 0.02175656156450629
exactly 25 : 0.030169098702782057
exactly 26 : 0.03983868162034041
exactly 27 : 0.05016722870709

0.3006022057910501